# ModOAP - Repérage de doublons images

Ce script permet de repérer des doublons au sein d'un corpus d'images.

Le corpus est attendu sous la forme d'un dossier sur un Google Drive contenant directement les fichiers images. 

Le script implémente la version CNN de la bibliothèque Imagededup : https://github.com/idealo/imagededup
(Tanuj Jain and Christopher Lennan and Zubin John and Dat Tran, 2019 )

**Ce carnet doit être exécuté dans un environnement GPU : Exécution -> Modifier le type d'exécution -> GPU**


In [1]:
#@markdown # Préparation du script et synchronisation d'un Google Drive

#@markdown Lancer cette cellule, puis cliquer sur le lien généré par Google pour connecter un compte Drive si demandé.

#@markdown Il est nécessaire de relancer l'environnement d'exécution et de relancer la cellule si précisé. 

# chargement d'un google drive

import os
from google.colab import drive

if not os.path.exists("/content/drive/My Drive"):
  drive.mount('/content/drive')
else : print("Le Drive est déjà monté")

try :
  from imagededup.methods import CNN, PHash, DHash, AHash, WHash
except ModuleNotFoundError :
  !pip install imagededup
  from imagededup.methods import CNN, PHash, DHash, AHash, WHash



Le Drive est déjà monté


In [ ]:
#@markdown # Repérage de doublons

#@markdown Cette cellule génère un fichier liste_doublons.txt dans le dossier du corpus



#@markdown Spécifier le chemin vers le dossier contenant les fichiers images :

chemin_corpus = "/content/drive/MyDrive/ModOAP/Outils_Modoap/Similarite/test"#@param {type:"string"}

#@markdown Spécifier le seuil de similarité attendu (un nombre décimal entre 0 et 1) :

seuil_similarite = 0.85 #@param {type:"slider", min:0, max:1, step:0.05}


cnn = CNN()
duplicates = cnn.find_duplicates(image_dir=chemin_corpus, scores=True, min_similarity_threshold=seuil_similarite)



with open(os.path.join(chemin_corpus,"liste_doublons.txt"), "w") as res :
  res.write("Doublons repérés dans le dossier {} \n".format(chemin_corpus))
  for entry, dups in duplicates.items() :
    if duplicates[entry]:
      res.write("---------------------------------------------\n")
      res.write("Image source :\n")
      res.write(entry+"\n")
      res.write("Doublons :\n")
      for dup in dups :
        res.write(dup[0]+"\n")
      res.write("---------------------------------------------\n")

print("La liste des doublons a été générée dans le fichier ",os.path.join(chemin_corpus,"liste_doublons.txt"))

In [ ]:
#@markdown # Visualisation des doublons

#@markdown Cette cellule permet de visualiser les doublons repérés

from imagededup.utils import plot_duplicates
for k in duplicates :
  try : 

    if duplicates[k] :
      print("Doublon repéré : ",k," -> ",duplicates[k])
    plot_duplicates(image_dir="/content/drive/MyDrive/ModOAP/Outils_Modoap/MonumenTal/testdedup",
                    duplicate_map=duplicates,
                    filename=k)
    print("-------------------------------------------")
  except AssertionError : 
    pass

## ***Encodage et calcul de distances spécifiques***

In [ ]:
#@markdown # Encodage des images du corpus

#@markdown Cette cellule génère un dictionnaire nommé encodings avec une clé par fichier image du corpus associée à son vecteur numpy

#@markdown Spécifier le chemin vers le dossier contenant les fichiers images (au format jpg ou changer le code) :

chemin_corpus = ""#@param {type:"string"}

encodings = CNN().encode_images(image_dir=chemin_corpus)

print("Le dictionnaire encodings a été généré")

In [ ]:
encodings

In [ ]:
import glob
from scipy.spatial import distance
from datetime import datetime

#@markdown # Calcul des distances 1

#@markdown Cacluler les distances entre les images au sein du corpus. 

#@markdown Cette cellule génère un fichier CSV avec les distances entre chaque image.

#@markdown Spécifier le type de calcul de la distance :
type_distance = "euclidean" #@param ["euclidean", "canberra", "cityblock", "cosine"]


now = datetime.now()
fichier_out = os.path.join(chemin_corpus,"resultat_sim_"+str(now)+"_"+type_distance+".csv").replace(" ","_")
with open(fichier_out, "w") as csv_out :
  csv_out.write("image_source;image_cible;distance_{}\n".format(type_distance))
  for k,v in encodings.items() :
    for k2,v2 in encodings.items() :
      if k != k2 :
        if type_distance == "cosine" :
          csv_out.write(str(k)+";"+str(k2)+";"+str(distance.cosine(v, v2))+"\n")
        elif type_distance == "euclidean" :
          csv_out.write(str(k)+";"+str(k2)+";"+str(distance.euclidean(v, v2))+"\n")
        if type_distance == "canberra" :
          csv_out.write(str(k)+";"+str(k2)+";"+str(distance.canberra(v, v2))+"\n")
        if type_distance == "cityblock" :
          csv_out.write(str(k)+";"+str(k2)+";"+str(distance.cityblock(v, v2))+"\n") 
print("Le fichier {} contenant les résultats a été généré".format(fichier_out))


In [ ]:
import glob
from scipy.spatial import distance
from datetime import datetime

#@markdown # Calcul des distances 2

#@markdown Cacluler les distances entre les images du corpus et les images d'un dossier extérieur au corpus. 

#@markdown Cette cellule génère un fichier CSV avec les distances entre chaque image.

#@markdown Spécifier le dossier contenant une ou plusieurs images extérieures au corpus :
corpus_ext = ""#@param {type:"string"}
#@markdown Spécifier le type de calcul de la distance :
type_distance = "euclidean" #@param ["euclidean", "canberra", "cityblock", "cosine"]
images_requetes = glob.glob(os.path.join(corpus_ext,"*.jpg"))

now = datetime.now()
fichier_out = os.path.join(corpus_ext,"resultat_sim_"+str(now)+"_"+type_distance+".csv").replace(" ","_")
with open(fichier_out, "w") as csv_out :
  csv_out.write("image_source;image_cible;distance_{}\n".format(type_distance))
  for img_req in images_requetes :
    encoding1 = CNN().encode_image(image_file=img_req)
    for k,v in encodings.items() :
      if type_distance == "cosine" :
        csv_out.write(str(img_req)+";"+str(k)+";"+str(distance.cosine(encoding1, v))+"\n")
      elif type_distance == "euclidean" :
        csv_out.write(str(img_req)+";"+str(k)+";"+str(distance.euclidean(encoding1, v))+"\n")
      if type_distance == "canberra" :
        csv_out.write(str(img_req)+";"+str(k)+";"+str(distance.canberra(encoding1, v))+"\n")
      if type_distance == "cityblock" :
        csv_out.write(str(img_req)+";"+str(k)+";"+str(distance.cityblock(encoding1, v))+"\n") 
print("Le fichier {} contenant les résultats a été généré".format(fichier_out))